In [1]:
##Cormac McHale

In [2]:
##Training the Neural Network

In [3]:
import gzip
import sklearn.preprocessing as pre
import keras as kr
import numpy as np
from keras.models import load_model

Using TensorFlow backend.


In [4]:
# Start a neural network, building it by layers.
model = kr.models.Sequential()

# Add a hidden layer with 1000 neurons and an input layer with 784.
model.add(kr.layers.Dense(units=1000, activation='relu', input_dim=784))
model.add(kr.layers.Dense(units=100, activation='sigmoid'))
model.add(kr.layers.Dense(units=1000, activation='relu'))
# Add a 10 neuron output layer for each digit??
model.add(kr.layers.Dense(units=10, activation='softmax'))

# Build the graph.
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [5]:
##file reading and formatting
imagesFile  = open('MNIST/train-images.idx3-ubyte', 'rb')
labelsFile = open('MNIST/train-labels.idx1-ubyte', 'rb')
images = imagesFile.read()
labels = labelsFile.read()
images =  (~np.array(list(images[16:])).reshape(60000, 28, 28).astype(np.uint8)-255)
labels =  np.array(list(labels[ 8:])).astype(np.uint8)

In [6]:
inputs = images.reshape(60000, 784)
inputs[inputs>0] = 1
inputs[inputs<1] = 0 

In [7]:
encoder = pre.LabelBinarizer()
encoder.fit(labels)
outputs = encoder.transform(labels)
#just for looking
#print(inputs[0])
#print (inputs[0].size)

In [8]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1000)              785000    
_________________________________________________________________
dense_2 (Dense)              (None, 100)               100100    
_________________________________________________________________
dense_3 (Dense)              (None, 1000)              101000    
_________________________________________________________________
dense_4 (Dense)              (None, 10)                10010     
Total params: 996,110
Trainable params: 996,110
Non-trainable params: 0
_________________________________________________________________


In [9]:
##change later to test
model.fit(inputs, outputs, epochs=5, batch_size=150)

Epoch 1/5
60000/60000 [==============================] - 9s 157us/step - loss: 0.3128 - accuracy: 0.9071
Epoch 2/5
60000/60000 [==============================] - 9s 152us/step - loss: 0.1128 - accuracy: 0.9661
Epoch 3/5
60000/60000 [==============================] - 9s 149us/step - loss: 0.0776 - accuracy: 0.9756
Epoch 4/5
60000/60000 [==============================] - 9s 153us/step - loss: 0.0574 - accuracy: 0.9822
Epoch 5/5
60000/60000 [==============================] - 9s 151us/step - loss: 0.0428 - accuracy: 0.9859


In [10]:
##need to then use the test images on the Neural Network...
imagesTest  = open('MNIST/t10k-images.idx3-ubyte', 'rb')
labelsTest = open('MNIST/t10k-labels.idx1-ubyte', 'rb')
testImages = imagesTest.read()
testLabels = labelsTest.read()

In [11]:
type(testImages)

bytes

In [12]:
testImages[0:4]

b'\x00\x00\x08\x03'

In [13]:
testImages = (~np.array(list(testImages[16:])).reshape(10000, 28, 28).astype(np.uint8)-255)
#print(testImages[0])
testLabels =  np.array(list(testLabels[ 8:])).astype(np.uint8)
testInputs = testImages.reshape(10000, 784)
testInputs[testInputs>0] = 1
testInputs[testInputs<1] = 0 
encoder = pre.LabelBinarizer()
encoder.fit(testLabels)
testOutputs = encoder.transform(testLabels)

In [14]:
# Have the network predict the classes of the test inputs.
predictions = model.predict(testInputs)
predictions = encoder.inverse_transform(predictions)
##Comparison
predictions == encoder.inverse_transform(testOutputs)
#check how 
(predictions == encoder.inverse_transform(testOutputs)).sum()

9689

In [15]:
#save the model
model.save('savedModel/my_model.h5')
##Initial Model finished... flask Next.. will return to this later
testInputs = testImages.reshape(10000, 784)
model.predict(testInputs)

array([[1.5133361e-07, 5.4020592e-07, 9.2213659e-06, ..., 9.9995530e-01,
        3.9286522e-08, 2.1520804e-05],
       [4.2956566e-10, 3.3887542e-07, 9.9999845e-01, ..., 6.1409516e-08,
        3.0408614e-08, 2.9077943e-11],
       [4.7331693e-07, 9.9983287e-01, 8.2523837e-07, ..., 2.9232941e-05,
        1.2992857e-04, 1.7475098e-06],
       ...,
       [1.0767602e-08, 2.4795000e-07, 4.9255431e-07, ..., 1.6476573e-06,
        5.9509102e-07, 5.8672862e-04],
       [1.5070017e-06, 2.0709874e-06, 1.5438924e-07, ..., 2.9009968e-07,
        9.8011238e-05, 1.2128787e-07],
       [3.3072538e-05, 1.3017734e-04, 5.7844715e-03, ..., 1.6013338e-06,
        1.4084880e-05, 8.8927745e-06]], dtype=float32)

In [16]:
##Model complete

In [17]:
##Re-train with an extra layer

In [18]:
type(model)

keras.engine.sequential.Sequential

In [19]:
fineTune = kr.models.Sequential()
for layer in model.layers:
    fineTune.add(layer)
for layer in fineTune.layers:
    layer.trainable = False

In [20]:
fineTune.add(kr.layers.Dense(units=10, activation='sigmoid'))

In [21]:
fineTune.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
fineTune.fit(inputs, outputs, epochs=10, batch_size=50)

Epoch 1/10
60000/60000 [==============================] - 4s 60us/step - loss: 0.3867 - accuracy: 0.9151
Epoch 2/10
60000/60000 [==============================] - 3s 56us/step - loss: 0.1481 - accuracy: 0.9939
Epoch 3/10
60000/60000 [==============================] - 3s 56us/step - loss: 0.0787 - accuracy: 0.9967
Epoch 4/10
60000/60000 [==============================] - 3s 57us/step - loss: 0.0478 - accuracy: 0.9970
Epoch 5/10
60000/60000 [==============================] - 3s 56us/step - loss: 0.0316 - accuracy: 0.9971
Epoch 6/10
60000/60000 [==============================] - 3s 56us/step - loss: 0.0225 - accuracy: 0.9971
Epoch 7/10
60000/60000 [==============================] - 3s 56us/step - loss: 0.0173 - accuracy: 0.9972
Epoch 8/10
60000/60000 [==============================] - 4s 64us/step - loss: 0.0143 - accuracy: 0.9972
Epoch 9/10
60000/60000 [==============================] - 4s 67us/step - loss: 0.0125 - accuracy: 0.9973
Epoch 10/10
60000/60000 [==============================

In [22]:
# Have the network predict the classes of the test inputs.
predictions = fineTune.predict(testInputs)
predictions = encoder.inverse_transform(predictions)
##Comparison
predictions == encoder.inverse_transform(testOutputs)
#check how 
(predictions == encoder.inverse_transform(testOutputs)).sum()

9700

In [23]:
#Results
#save new model
fineTune.save('savedModel/freshlyTrained.h5')
#test in client